# Выбор локации для скважины

Нужно построить модель машинного обучения для определения региона, где добыча нефти принесет наибольшую прибыль. Для решения задачи предоставлены пробы нефти в трех регионах, в каждом 10 000 месторождений, где было измерено качество нефти и объем ее запасов. Необходимо проанализировать прибыль и риски техникой Bootstrap.

Условия задачи:
- для обучения модели выбрать линейную регрессию;
- при разведке региона исследовать 500 точек, из которых с помощью машинного обучения выбрать 200 лучших для разработки;
- бюджет на разработку скважин в регионе — 10 млрд рублей;
- один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей;
- после оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбрать регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [1]:
from dataclasses import dataclass, asdict

import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [2]:
raw_datasets = []

for i in range(3):
    geo_data = pd.read_csv(f'/datasets/geo_data_{i}.csv')
    raw_datasets.append(geo_data)
    print(f'\nДанные геологической разведки региона {i}')
    display(geo_data.head())
    display(geo_data.describe())
    geo_data.info()


Данные геологической разведки региона 0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Данные геологической разведки региона 1


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Данные геологической разведки региона 2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Данные геологоразведки трёх регионов содержат следующие столбцы:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (не важно их обозначение, важна значимость признаков);
- product — объём запасов в скважине (тыс. баррелей).

**Вывод:** загружены три датасета `geo_data_0`, `geo_data_1`, `geo_data_2` о геологической разведке трех регионов, все таблицы содержат по 10000 строк с месторождениями нефти и 5 столбцов. Пропущенных значений не обнаружено, датасет содержит два типа данных: `object`, `float`. Столбцы `f0`, `f1`, `f2` содержат значимые признаки, но неизвестно, что они означают, поэтому я не буду приводить их к целочисленному типу, чтобы не потерять точность данных. А так же столбец `product` оставлю с типом данных `float`, чтобы объем запасов в скважине оставить с максимальной точностью (для дальнейшего вычисления прибыли). В данных аномалий не обнаружено, поэтому предобработка не требуется.

Для дальнейшего обучения модели столбец `id` с уникальным идентификатором скважины не понадобится, поэтому удалю его из трех датасетов.

In [3]:
for data in raw_datasets:
    data.drop(['id'], axis=1, inplace=True)
    display(data.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


## Обучение и проверка модели

### Разделение данных на выбоки

Целевым признаком является объем запасов в скважине, его определю в переменную `target`. Остальные признаки - в переменную `features`.

Разделю каждый датасет на две выборки:
- обучающую, на которой соответственно будут обучаться модели. Целевой признак положу в переменную `target_train`. Остальные признаки запишу в переменную `features_train`;
- валидационную, для оценки качества модели (признаки - `features_valid`, целевой признак - `target_valid`).

Обучающая и валидационная выборки делятся в соотношении 3:1 с помощью функции библиотеки sklearn `train_test_split`. Для удобства все переменные оформлю в датакласс `Sample`.

In [4]:
@dataclass
class Sample:
    features_train: DataFrame
    features_valid: DataFrame
    target_train: Series
    target_valid: Series

In [5]:
samples = []

for data in raw_datasets:
    features = data.drop(['product'], axis=1)
    target = data['product']
    
    splitted_data = train_test_split(
        features, target, test_size=0.25, random_state=123
    )
    sample = Sample(*splitted_data)
    samples.append(sample)

Для проверки выведу размеры полученных наборов:

In [6]:
for name, dataframe in asdict(samples[0]).items():
    print(name, dataframe.shape)

features_train (75000, 3)
features_valid (25000, 3)
target_train (75000,)
target_valid (25000,)


### Обучение модели линейной регрессии

In [7]:
products = []

for number, data in enumerate(samples):
    model = LinearRegression()
    model.fit(data.features_train, data.target_train)
    predicted_valid = model.predict(data.features_valid)
    mse = mean_squared_error(data.target_valid, predicted_valid)
    
    products.append({
        "predicted": pd.Series(predicted_valid, index=data.target_valid.index),
        "target": data.target_valid
    })
    
    print(f"Регион {number}")
    print("Средний запас предсказанного сырья:", predicted_valid.mean())
    print("RMSE:", mse ** 0.5)

Регион 0
Средний запас предсказанного сырья: 92.54936189116306
RMSE: 37.64786282376176
Регион 1
Средний запас предсказанного сырья: 69.28001860653976
RMSE: 0.8954139804944313
Регион 2
Средний запас предсказанного сырья: 95.09859933591373
RMSE: 40.12803006598514


### Оценка качества модели кросс-валидацией

In [8]:
for data in raw_datasets:
    features = data.drop(['product'], axis=1)
    target = data['product']
    scores = cross_val_score(model, features, target, cv=5)
    final_score = sum(scores) / len(scores)
    print('Средняя оценка качества модели:', final_score)

Средняя оценка качества модели: 0.27549130726904475
Средняя оценка качества модели: 0.9996243728923553
Средняя оценка качества модели: 0.1987156246205129


**Вывод:** по полученным результатам видно, что средний объем сырья для 1 региона сильно разнится со средним значением сырья для 0 и 2 регионов. Также можно сказать, что для 1 региона модель лучше обучилась, чем для других, о чем говорит значение метрики RMSE, а также средняя оценка качества модели.

## Подготовка к расчёту прибыли

Посчитаю объем сырья, который нужно добыть в одном регионе, чтобы выйти на безубыточную добычу:

In [9]:
EXPENSES = 10*10**9
INCOME_PER_PRODUCT = 450000
target_product = EXPENSES / INCOME_PER_PRODUCT
target_product

22222.222222222223

Посчитаю средний объем сырья, который нужно добыть в одной скважине:

In [10]:
WELLS = 200
mean_product = target_product / WELLS
mean_product

111.11111111111111

**Вывод:** средний объем сырья для безубыточной добычи больше, чем средний объем, предсказанный моделью для каждого региона. Это еще раз подтверждает необходимость выбирать 200 скважин с наибольшим объемом запасов.

## Расчёт прибыли и рисков 

In [11]:
"""Функция для расчета прибыли"""

def calc_profit(predicted, target, count):
    best_predicted = predicted.sort_values(ascending=False)
    unique_indexes = best_predicted.index.unique()
    best_product = target[unique_indexes].iloc[:count]
    return best_product.sum() * INCOME_PER_PRODUCT - EXPENSES

Для каждого региона выведу значение прибыли в млрд.руб:

In [12]:
for number, value in enumerate(products):
    profit = calc_profit(value["predicted"], value["target"], WELLS)
    print(
        f'Прибыль при добыче в регионе {number}:', 
        round(profit / 10**9, 3), 
        'млрд.руб.'
    )

Прибыль при добыче в регионе 0: 3.535 млрд.руб.
Прибыль при добыче в регионе 1: 2.415 млрд.руб.
Прибыль при добыче в регионе 2: 2.37 млрд.руб.


In [13]:
state = RandomState(123)

Посчитаю распределение прибыли и риски с помощью техники bootstrap:

In [14]:
for number, value in enumerate(products):
    values = []
    for i in range(1000):
        target_sample = value["target"].sample(n=500, replace=True, random_state=state)
        predicted_sample = value["predicted"][target_sample.index]
        profit = calc_profit(predicted_sample, target_sample, WELLS)
        values.append(profit)

    values = pd.Series(values)
    lower = values.quantile(q=0.025) 
    upper = values.quantile(q=0.975)

    print(f"\nРегион {number}:")
    print(
        "95-ти процентный доверительный интервал:", 
        round(lower / 10**9, 3), 
        "-", 
        round(upper / 10**9, 3), 
        "млрд.руб."
    )
    print("Средняя прибыль:", round(values.mean() / 10**9, 3), "млрд.руб.") 
    print("Риск убытков:", round((values < 0).mean() * 100, 2), "%")


Регион 0:
95-ти процентный доверительный интервал: -0.058 - 0.975 млрд.руб.
Средняя прибыль: 0.477 млрд.руб.
Риск убытков: 4.1 %

Регион 1:
95-ти процентный доверительный интервал: 0.069 - 0.871 млрд.руб.
Средняя прибыль: 0.467 млрд.руб.
Риск убытков: 1.0 %

Регион 2:
95-ти процентный доверительный интервал: -0.183 - 0.871 млрд.руб.
Средняя прибыль: 0.362 млрд.руб.
Риск убытков: 8.9 %


**Выбор наилучшего региона для бурения скважин:** 
- по условию задачи, риск убытков должен быть менее 2.5%, поэтому регионы 0 и 2 исключаются из рассмотрения, также у этих регионов нижнее значение доверительного 95-ти процентного интервала отрицательное;

- по предсказанию модели у региона 0 прибыль составляет 3,5 млрд.руб., а у региона 1 - 2,4 млрд.руб. Разница по прибыли значительная, но при этом риск убытков у региона 0 больше на 3.1%.

**Условия задачи ограничивают по допустимой вероятности рисков, поэтому наилучшим регионом для бурения скважин выбирается регион 1 с максимальной прибылью по предсказанию модели 2.4 млрд.руб. с возможными рисками убытков 1.0%.**